 Install a required library into your notebook environment by running this code in a cell in your notebook:

In [ ]:
!pip install --upgrade wget

Download the sample model-building code to your notebook working directory by running this code in a cell in your notebook:

In [ ]:
import os
import wget
filename = 'tf-model.zip'
url = 'https://github.com/pmservice/wml-sample-models/blob/master/tensorflow/hand-written-digit-recognition/definition/tf-model.zip?raw=true'
if not os.path.isfile( filename ): wget.download( url )


In this tutorial, you don't train the model by running the sample model-building code in your notebook directly. Instead, you submit a job to run the model-building code in a training run on Watson Machine Learning specialized, high-performance infrastructure.

Submitting a training run requires two things:

A .zip file containing model-building code
Metadata
When using the Python client, you can specify the metadata in your Python code, or in a training run manifest file. This tutorial demonstrates specifying metadata in the Python code. The CLI MNIST tutorial demonstrates specifying training run metadata in a manifest file.

Terminology note: The model-building code and the metadata together are referred to as a training definition. To keep those terms straight, you can think of "training definition" as being short for "training run definition".

Install the Watson Machine Learning python client library into your notebook environment by running this code in a cell in your notebook:

In [ ]:
!pip install watson-machine-learning-client


Instantiate a client object by running this code in a cell in your notebook:

Note: Replace *** here with the Watson Machine Learning service credentials you collected as a prerequisite at the beginning of this tutorial. These are not the Cloud Object Storage service credentials you created earlier.


In [ ]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient
wml_credentials = { "url"         : "https://ibm-watson-ml.mybluemix.net",
                    "username"    : "***",
                    "password"    : "***",
                    "instance_id" : "***"
                   }
client = WatsonMachineLearningAPIClient( wml_credentials )


Store the training definition in the Watson Machine Learning repository by running this code in a cell in your notebook:

Note: Replace *** with your IBMid email.

In [ ]:
metadata = {
client.repository.DefinitionMetaNames.NAME              : "python-client-tutorial_training-definition",
client.repository.DefinitionMetaNames.AUTHOR_EMAIL      : "***",
client.repository.DefinitionMetaNames.FRAMEWORK_NAME    : "tensorflow",
client.repository.DefinitionMetaNames.FRAMEWORK_VERSION : "1.5",
client.repository.DefinitionMetaNames.RUNTIME_NAME      : "python",
client.repository.DefinitionMetaNames.RUNTIME_VERSION   : "3.5",
client.repository.DefinitionMetaNames.EXECUTION_COMMAND : "python3 convolutional_network.py --trainImagesFile ${DATA_DIR}/train-images-idx3-ubyte.gz --trainLabelsFile ${DATA_DIR}/train-labels-idx1-ubyte.gz --testImagesFile ${DATA_DIR}/t10k-images-idx3-ubyte.gz --testLabelsFile ${DATA_DIR}/t10k-labels-idx1-ubyte.gz --learningRate 0.001 --trainingIters 20000"
}
definition_details = client.repository.store_definition( "tf-model.zip", meta_props=metadata )
definition_uid     = client.repository.get_definition_uid( definition_details )
print( "definition_uid: ", definition_uid )


Define training run metadata by running this code in a cell in your notebook:

Note: Replace *** here with your IBMid email, the bucket names and endpoint URLs of the buckets you created earlier, as well as the Cloud Object Storage service credentials you created earlier.
These are the access_key_id, the secret_access_key, and the resource_instance_id from the credentials created earlier.

In [ ]:
metadata = {
client.training.ConfigurationMetaNames.NAME         : "python-client-tutorial_training-run",
client.training.ConfigurationMetaNames.AUTHOR_EMAIL : "***",
client.training.ConfigurationMetaNames.TRAINING_DATA_REFERENCE : {
   "connection" : { 
      "endpoint_url"      : "https://s3.us-south.cloud-object-storage.appdomain.cloud",
      "access_key_id"     : "***",
      "secret_access_key" : "***"
      },
   "source" : { 
      "bucket" : "techtogether-test-data2",
      },
      "type" : "s3"
   },
client.training.ConfigurationMetaNames.TRAINING_RESULTS_REFERENCE: {
   "connection" : {
      "endpoint_url"      : "https://s3.us-south.cloud-object-storage.appdomain.cloud",
      "access_key_id"     : "***",
      "secret_access_key" : "***"
      },
      "target" : {
         "bucket" : "techtogether-test-training",
      },
      "type" : "s3"
   }
}


Start the training run by running this code in a cell in your notebook:

In [ ]:
run_details = client.training.run( definition_uid, meta_props=metadata )
run_uid     = client.training.get_run_uid( run_details )
print( "run_uid: ", run_uid )


You can monitor the progress of a training run by running this code in a cell in your notebook:

In [ ]:
client.training.get_status( run_uid )

After the training run finishes, you can view log files and other output in the training results bucket of your Cloud Object Storage.
See: Viewing results in Cloud Object Storage


# Deploy the trained model
You can use your trained model to classify new images only after the model has been deployed.

Before you can deploy the model, you must store the trained model in the Watson Machine Learning repository. Store the trained model by running this code in a cell in your notebook:

In [ ]:
stored_model_name    = "python-client-tutorial_model"
stored_model_details = client.repository.store_model( run_uid, stored_model_name )
model_uid            = client.repository.get_model_uid( stored_model_details )
print( "model_uid: ", model_uid )


Deploy the stored model by running this code in a cell in your notebook:

In [ ]:
deploymnt_name   = "MNIST-handwriting"
deployment_desc  = "Online deployment of Python client tutorial model"
deployment       = client.deployments.create( model_uid, deploymnt_name, deployment_desc )
scoring_endpoint = client.deployments.get_scoring_url( deployment )
print( "scoring_endpoint: ", scoring_endpoint )


# Test the deployed model

Download a sample file with images of the handwritten digits "5" and "4" by running this code in a cell in your notebook:

In [ ]:
filename = 'tf-mnist-test-payload.json'
url = 'https://raw.githubusercontent.com/pmservice/wml-sample-models/master/tensorflow/hand-written-digit-recognition/test-data/tf-mnist-test-payload.json'
if not os.path.isfile( filename ): wget.download( url )

Load the contents of the file into a structure by running this code in a cell in your notebook:

In [ ]:
import json
with open( 'tf-mnist-test-payload.json' ) as data_file: test_data = json.load( data_file )
payload = test_data[ 'payload' ]

Classify the images in the test data by running this code in a cell in your notebook:

In [ ]:
client.deployments.score( scoring_endpoint, payload )

The results should be:

In [ ]:
{'values': [5, 4]}